### Importing initial modules

In [1]:
import os
import numpy as np
import pandas as pd

In [3]:
os.getcwd()

'C:\\Users\\baari\\Documents\\uni\\y3\\project\\final_project\\hard-drive-predictive-maintenance'

In [ ]:
df = pd.read_csv('Q1_2019.csv')

In [ ]:
df.shape

In [ ]:
df.head()

In [ ]:
df.tail()

In [ ]:
df['failure'].value_counts()

### Handling missing values

In [ ]:
total_cells = np.prod(df.shape)
total_cells

In [ ]:
missing_values_count = df.isnull().sum()
missing_values_count[0:10]

In [ ]:
total_missing = missing_values_count.sum()
total_missing

In [ ]:
percent_missing = total_missing / total_cells * 100
print(f'{percent_missing}% of the data is missing')

In [ ]:
columns_before = df.columns

# Remove columns that have all null values
df = df.dropna(axis=1, how='all')

columns_after = df.columns

removed_columns = set(columns_before) - set(columns_after)
print(f'Columns removed: {removed_columns}')

In [ ]:
total_cells = np.prod(df.shape)
total_missing = df.isnull().sum().sum()
percent_missing = total_missing / total_cells * 100
print(f'{percent_missing}% of the data is missing')